In [5]:
import pandas as pd
import os
import librosa
import numpy as np
import math
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
import librosa.display

In [6]:
audio_dataset_path='C:/Users/aleem/Desktop/Currentlyworking_New_extracted audio wav after cleaning then augmentation/sliding_window_1_sec/data/Al-Masad/'
metadata=pd.read_csv('C:/Users/aleem/Desktop/Currentlyworking_New_extracted audio wav after cleaning then augmentation/sliding_window_1_sec/data/Al-Masad/metadata_111.csv')
metadata.head(20)

,slice_file_name,fold,classID,class
0,3_1.wav,1,111,Al-Masad
1,3_2.wav,1,111,Al-Masad
2,3_3.wav,1,111,Al-Masad
3,3_4.wav,1,111,Al-Masad
4,3_5.wav,1,111,Al-Masad
5,2_1.wav,2,111,Al-Masad
6,2_2.wav,2,111,Al-Masad
7,2_3.wav,2,111,Al-Masad
8,2_4.wav,2,111,Al-Masad
9,2_5.wav,2,111,Al-Masad


Feature Extraction

In [7]:
def mfcc_features_extractor_1sec(file):
    # audio, sample_rate = librosa.load(file_name) 
    # mfccs_features = librosa.feature.mfcc(y=audio, sr=16000, n_mfcc=40)
    # mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    y, sr = librosa.load(file, sr=16000)
    
    y = y[:192000]  #12 Secs for all audios
    zero_padding = np.zeros(192000 - y.shape[0], dtype=np.float32)
    y = np.concatenate([y, zero_padding])   

    # Define window size and hop length
    win_length = int(sr * 1) # 1 second window
    hop_length = int(win_length / 2) # 50% overlap

    # Extract MFCC features for each window
    mfccs = []
    for i in range(0, len(y)-win_length, hop_length):
        mfcc = librosa.feature.mfcc(y[i:i+win_length], sr=sr, n_mfcc=40)
        mfccs.append(mfcc)

    # Combine MFCC features for all windows
    mfccs = np.concatenate(mfccs, axis=1)
    mfccs = mfccs.T

    # Return final MFCC features
    return mfccs

    
    # return mfccs_scaled_features

In [8]:
# mfcc_extracted_features=[]

# for index_num,row in tqdm(metadata.iterrows()):
#     file_name = os.path.join(os.path.abspath(audio_dataset_path),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
#     final_class_labels=row["class"]
    
#     # MFCC Features Extraction
#     mfcc_data= mfcc_features_extractor_1sec(file_name)
#     mfcc_extracted_features.append([mfcc_data,final_class_labels])
#     # print(mfcc_extracted_features)

In [9]:
mfcc_data_by_fold = {}

# Iterate through the DataFrame and extract MFCC features
for index_num, row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path), 'fold' + str(row["fold"]) + '/', str(row["slice_file_name"]))
    final_class_labels = row["class"]

    # MFCC Features Extraction
    mfcc_data = mfcc_features_extractor_1sec(file_name)
    
    # Store the MFCC data in the dictionary by fold number
    fold_num = row["fold"]
    if fold_num not in mfcc_data_by_fold:
        mfcc_data_by_fold[fold_num] = {
            "data": [],
            "labels": []
        }
    mfcc_data_by_fold[fold_num]["data"].append(mfcc_data)
    mfcc_data_by_fold[fold_num]["labels"].append(final_class_labels)

# Now, convert the data in mfcc_data_by_fold to NumPy arrays
for fold_num, data_dict in mfcc_data_by_fold.items():
    data_dict["data"] = np.array(data_dict["data"])
    data_dict["labels"] = np.array(data_dict["labels"])

0it [00:00, ?it/s]C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[ 0.00011151 -0.00042198 -0.00073662 ...  0.01680131 -0.04821217
 -0.09365726] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[ 0.24846943  0.25380632  0.23881698 ... -0.00435433 -0.07913796
 -0.13074403] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-0.11722073 -0.1317492  -0.1295128  ...  0.07969461  0.06524467
  0.10849177] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-0.18015479 -0.23

In [10]:
fold_num = 1  # Replace with the fold number you're interested in
data_shape = mfcc_data_by_fold[fold_num]["data"].shape
labels_shape = mfcc_data_by_fold[fold_num]["labels"].shape

print(f"Shape of data for fold {fold_num}: {data_shape}")
print(f"Shape of labels for fold {fold_num}: {labels_shape}")

Shape of data for fold 1: (5, 704, 40)
Shape of labels for fold 1: (5,)


In [11]:
mfcc_data_by_fold[3]["data"].shape

(5, 704, 40)

In [12]:
# Iterate through the dictionary and reshape the data arrays for all fold_num values
for fold_num, data_dict in mfcc_data_by_fold.items():
    data_dict["data"] = data_dict["data"].reshape(data_dict["data"].shape + (1,))
    

In [13]:
mfcc_data_by_fold[1]["data"].shape

(5, 704, 40, 1)

After Training Work, Deep Features 

In [14]:
from keras.models import load_model
from keras.models import Model

# Load the saved model
loaded_model = load_model('Model_CNN2D_startwithnumpylengthsamemfcc_1sec_4032files_90_10_size_result 100_96.29.h5')

# Get the name of the intermediate layer from the loaded model summary
intermediate_layer_name = 'dense'

# Find the index of the intermediate layer in the model's layers list
intermediate_layer_index = [i for i, layer in enumerate(loaded_model.layers) if layer.name == intermediate_layer_name][0]

# Define a new model that includes layers up to the intermediate layer
intermediate_layer_model = Model(inputs=loaded_model.input,
                                  outputs=loaded_model.layers[intermediate_layer_index].output)

# List of inputs for each fold's MFCC data
input_data_list = [mfcc_data_by_fold[fold]["data"] for fold in range(1, 4)]  # Adjust the range as needed

# List to store intermediate layer outputs for each fold
intermediate_layer_outputs = []

# Iterate through the input data for each fold and predict the intermediate layer output
for input_data in input_data_list:
    intermediate_output = intermediate_layer_model.predict(input_data)
    intermediate_layer_outputs.append(intermediate_output)

# Now, intermediate_layer_outputs contains the intermediate layer outputs for each fold's input data


1/1 [==============================] - 0s 73ms/step


In [15]:
intermediate_layer_outputs = np.array(intermediate_layer_outputs)

In [16]:
intermediate_layer_outputs.shape

(3, 5, 1024)

In [17]:
intermediate_layer_outputs

array([[[-0.9998462 , -1.        ,  0.99999785, ...,  0.99932086,
          0.9874426 , -0.9957767 ],
        [-0.99999917, -1.        ,  1.        , ...,  0.99991006,
         -0.96371   , -0.99996734],
        [-0.99704134, -1.        ,  0.9998545 , ...,  0.9951254 ,
         -0.8974858 , -0.9973104 ],
        [-0.99985147, -1.        ,  0.99996704, ...,  0.99826306,
         -0.99018997, -0.9984771 ],
        [-0.9994696 , -1.        ,  0.9999971 , ...,  0.99889916,
         -0.8722929 , -0.9999312 ]],

       [[-0.9997904 , -1.        ,  1.        , ...,  0.9991492 ,
         -0.9999165 , -1.        ],
        [-0.99990493, -0.99815965,  0.9999989 , ...,  0.9858401 ,
         -0.99360746, -0.99964476],
        [-0.9993705 , -1.        ,  0.99999654, ...,  0.9860347 ,
         -0.36746603, -0.9990431 ],
        [-0.99821335, -0.999998  ,  0.99993336, ...,  0.9883784 ,
         -0.98695076, -0.9999904 ],
        [-0.9959029 , -0.76434326,  1.        , ...,  0.9864085 ,
         -0.98

In [18]:
mean_across_folds = np.mean(intermediate_layer_outputs, axis=0)

In [19]:
al_masad_mean = mean_across_folds

In [20]:
al_masad_mean.shape

(5, 1024)

In [21]:
al_masad_mean

array([[-0.98590374, -0.3334246 ,  0.99450463, ...,  0.99900466,
        -0.33182678, -0.90482444],
       [-0.979811  , -0.33273456,  0.6230114 , ...,  0.9879365 ,
        -0.6291113 , -0.96305627],
       [-0.93836504, -0.9997275 ,  0.977863  , ...,  0.89121056,
        -0.20281863, -0.9892564 ],
       [-0.99820787, -0.896251  ,  0.9969563 , ...,  0.99541086,
        -0.9543262 , -0.9994822 ],
       [-0.95316315, -0.59610826,  0.9949455 , ...,  0.99276906,
        -0.7833721 , -0.98554724]], dtype=float32)

In [22]:
# Specify the file path where you want to save the .npy file
file_path = 'al_masad_mean.npy'

# Save the array as a .npy file
np.save(file_path, al_masad_mean)

Testing of Single Audio

In [23]:
file_name = 'C:/Users/aleem/Desktop/Currentlyworking_New_extracted audio wav after cleaning then augmentation/sliding_window_1_sec/data/masad_testing.wav'


mfcc_extracted_features_single=[]

mfcc_extracted_features_single= mfcc_features_extractor_1sec(file_name)

mfcc_extracted_features_single = np.array(mfcc_extracted_features_single)

mfcc_extracted_features_single = mfcc_extracted_features_single.reshape((1, 704, 40, 1))



C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-0.00161382 -0.00183711 -0.00054128 ...  0.07860803  0.06741416
  0.08201688] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[0.34573337 0.28929874 0.26712272 ... 0.21383868 0.25290594 0.23459184] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[ 0.0285759  -0.0363187  -0.0368802  ... -0.19760664 -0.1702978
 -0.16580413] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[0.21447808 0.2388341  0.20046939 ... 0.0318

In [24]:
mfcc_extracted_features_single

array([[[[ -478.67667  ],
         [   51.20242  ],
         [   -7.2791033],
         ...,
         [    9.5946865],
         [    2.0298865],
         [    7.0562334]],

        [[ -462.3167   ],
         [   60.030037 ],
         [  -11.425642 ],
         ...,
         [    5.861943 ],
         [    2.0790195],
         [   10.7699795]],

        [[ -464.2313   ],
         [   56.99457  ],
         [   -9.574049 ],
         ...,
         [    3.5644107],
         [    1.8112628],
         [   13.666845 ]],

        ...,

        [[-1131.3708   ],
         [    0.       ],
         [    0.       ],
         ...,
         [    0.       ],
         [    0.       ],
         [    0.       ]],

        [[-1131.3708   ],
         [    0.       ],
         [    0.       ],
         ...,
         [    0.       ],
         [    0.       ],
         [    0.       ]],

        [[-1131.3708   ],
         [    0.       ],
         [    0.       ],
         ...,
         [    0.       ],
        

In [25]:
from keras.models import load_model
from keras.models import Model

# Load the saved model
loaded_model = load_model('Model_CNN2D_startwithnumpylengthsamemfcc_1sec_4032files_90_10_size_result 100_96.29.h5')

# Get the name of the intermediate layer from the loaded model summary
intermediate_layer_name = 'dense'

# Find the index of the intermediate layer in the model's layers list
intermediate_layer_index = [i for i, layer in enumerate(loaded_model.layers) if layer.name == intermediate_layer_name][0]

# Define a new model that includes layers up to the intermediate layer
intermediate_layer_model = Model(inputs=loaded_model.input,
                                  outputs=loaded_model.layers[intermediate_layer_index].output)

# Assuming you already have mfcc_extracted_features of shape (704, 40, 1)
# Replace this with your actual input data

# Predict the intermediate layer output for the input data
intermediate_output = intermediate_layer_model.predict(mfcc_extracted_features_single)

# Now, intermediate_output contains the intermediate layer output for mfcc_extracted_features


1/1 [==============================] - 0s 156ms/step


In [26]:
intermediate_output

array([[-0.9965587, -0.6887548,  0.9909684, ...,  0.999591 , -0.885838 ,
        -0.9999791]], dtype=float32)

In [27]:
intermediate_output.shape

(1, 1024)

In [28]:
intermediate_layer_outputs

array([[[-0.9998462 , -1.        ,  0.99999785, ...,  0.99932086,
          0.9874426 , -0.9957767 ],
        [-0.99999917, -1.        ,  1.        , ...,  0.99991006,
         -0.96371   , -0.99996734],
        [-0.99704134, -1.        ,  0.9998545 , ...,  0.9951254 ,
         -0.8974858 , -0.9973104 ],
        [-0.99985147, -1.        ,  0.99996704, ...,  0.99826306,
         -0.99018997, -0.9984771 ],
        [-0.9994696 , -1.        ,  0.9999971 , ...,  0.99889916,
         -0.8722929 , -0.9999312 ]],

       [[-0.9997904 , -1.        ,  1.        , ...,  0.9991492 ,
         -0.9999165 , -1.        ],
        [-0.99990493, -0.99815965,  0.9999989 , ...,  0.9858401 ,
         -0.99360746, -0.99964476],
        [-0.9993705 , -1.        ,  0.99999654, ...,  0.9860347 ,
         -0.36746603, -0.9990431 ],
        [-0.99821335, -0.999998  ,  0.99993336, ...,  0.9883784 ,
         -0.98695076, -0.9999904 ],
        [-0.9959029 , -0.76434326,  1.        , ...,  0.9864085 ,
         -0.98

Euclidian Distance with Mean

In [29]:
import numpy as np

# Assuming you have al_fil_mean with shape (5, 1024) and intermediate_output with shape (1, 1024)
# Calculate the Euclidean distances between intermediate_output and each row in al_fil_mean
distances = np.linalg.norm(al_masad_mean - intermediate_output, axis=1)

# Find the index of the closest row in al_fil_mean
closest_index = np.argmin(distances)

# The value at closest_index in al_fil_mean is the closest to intermediate_output
closest_value = al_masad_mean[closest_index]

# Print the closest_index and closest_value
print(f"Index of closest value in al_masad_mean: {closest_index+1}")
print(f"Closest value in al_masad_mean: {closest_value}")

Index of closest value in al_masad_mean: 4
Closest value in al_masad_mean: [-0.99820787 -0.896251    0.9969563  ...  0.99541086 -0.9543262
 -0.9994822 ]


Euclidian Distance Without Mean

In [30]:
# # Reshape intermediate_output to match the shape of intermediate_layer_outputs
# intermediate_output_reshaped = intermediate_output.reshape((1, 1, 1024))

# # Initialize variables to keep track of the closest indices and distances
# closest_indices = None
# min_distances = None
# closest_set = None  # Initialize variable to track the closest set

# for i, intermediate_set in enumerate(intermediate_layer_outputs):
#     # Calculate the Euclidean distances between intermediate_output and intermediate_set
#     distances = np.linalg.norm(intermediate_set - intermediate_output_reshaped, axis=2)
    
#     # Find the index of the minimum distance
#     current_closest_index = np.argmin(distances)
    
#     # Update the closest index, distances, and closest set if this set is closer
#     if closest_indices is None or np.mean(distances) < np.mean(min_distances):
#         closest_indices = current_closest_index
#         min_distances = distances
#         closest_set = i

# # Print the closest index in intermediate_layer_outputs and which set it belongs to
# print(f"Closest index in intermediate_layer_outputs: {closest_indices}")
# if closest_set == 0:
#     print("It is closest to intermediate_layer_outputs[0]")
# elif closest_set == 1:
#     print("It is closest to intermediate_layer_outputs[1]")
# elif closest_set == 2:
#     print("It is closest to intermediate_layer_outputs[2]")